# 학습1 - Model 1 (번개장터 데이터로 학습)

In [ ]:
# GPU 에러 보기 위함

import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ['TORCH_USE_CUDA_DSA'] = '1'

# 환경 설정

In [ ]:
# 통합코드 (GPT)

!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip3 install kobert-transformers

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_transformers import get_kobert_model, get_tokenizer

device = torch.device("cuda:0") # GPU
# device = torch.device("cpu") # CPU
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = get_kobert_model()
tokenizer = get_tokenizer() # vocab 역할

# 이후부터 모델 및 토크나이저를 사용하여 작업 진행.


In [ ]:
# device = torch.device("cuda:0") # GPU
# device = torch.device("cpu") # CPU

# 데이터셋 전처리

In [2]:
# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/sample_bj.csv')

In [ ]:
data

,id,name,cat_id
0,207604546,아쿠비클럽 숏패딩,310300200
1,172324242,듀베티카 이탈리아 브랜드 경량 패딩,310300200
2,216829622,"자라 러버라이즈드 패딩점퍼 xs 샌드(세탁1회가 전부,착용안함,새상품)",310300200
3,208445966,아이보리 퀼팅 패딩,310300200
4,214399932,Kenneth Cole 케네스콜 미들 다운패딩 S,310300200
...,...,...,...
156995,217819703,"플라워, 곰 키링, 키홀더, 열쇠고리",400999
156996,174817306,쿠로미핀,400999
156997,225929172,지방시 U 패드락,400999
156998,204684604,고양이 쿠키 브로치/페이크스위츠/로리타/카와이스타일/미니어쳐/스위티시모,400999


In [ ]:
# 데이터 전처리


In [4]:
# 번개장터 카테고리id - 카테고리명 매칭 작업

# cate = pd.read_excel('/content/drive/MyDrive/KDT/Final Project/data/categroy.xlsx', engine='openpyxl')
# cate = cate.rename(columns={'대분류':'cat1', '중분류':'cat2', '소분류':'cat3'})
cate = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/cate.csv')
cate

,id,cat1,cat2,cat3
0,310,여성의류,NaN,NaN
1,320,남성의류,NaN,NaN
2,400,패션 액세서리,NaN,NaN
3,405,신발,NaN,NaN
4,421,시계,NaN,NaN
...,...,...,...,...
194,430200400,가방/지갑,남성가방,백팩
195,430200500,가방/지갑,남성가방,브리프케이스
196,430200999,가방/지갑,남성가방,기타 남성가방
197,430300100,가방/지갑,여행용 가방,캐리어


In [5]:
cate.reset_index(inplace=True)
cate.rename(columns={'index': 'cate_enco'}, inplace=True)
cate

,cate_enco,id,cat1,cat2,cat3
0,0,310,여성의류,NaN,NaN
1,1,320,남성의류,NaN,NaN
2,2,400,패션 액세서리,NaN,NaN
3,3,405,신발,NaN,NaN
4,4,421,시계,NaN,NaN
...,...,...,...,...,...
194,194,430200400,가방/지갑,남성가방,백팩
195,195,430200500,가방/지갑,남성가방,브리프케이스
196,196,430200999,가방/지갑,남성가방,기타 남성가방
197,197,430300100,가방/지갑,여행용 가방,캐리어


In [ ]:
len(cate)

199

In [6]:
# 원래 카테고리id와 라벨인코딩한 카테고리id 저장해놓은 딕셔너리

cate_num = {}
for idx, row in cate.iterrows():
    cate_num[row['id']] = row['cate_enco']
cate_num

{310: 0,
 320: 1,
 400: 2,
 405: 3,
 421: 4,
 422: 5,
 430: 6,
 310120: 7,
 310130: 8,
 310150: 9,
 310200: 10,
 310220: 11,
 310250: 12,
 310260: 13,
 310300: 14,
 310400: 15,
 320120: 16,
 320160: 17,
 320180: 18,
 320210: 19,
 320300: 20,
 320400: 21,
 320500: 22,
 400070: 23,
 400080: 24,
 400090: 25,
 400110: 26,
 400120: 27,
 400130: 28,
 400140: 29,
 400600: 30,
 400999: 31,
 405100: 32,
 405200: 33,
 405300: 34,
 405400: 35,
 421100: 36,
 421200: 37,
 421300: 38,
 422100: 39,
 422200: 40,
 422300: 41,
 422400: 42,
 422500: 43,
 422600: 44,
 422999: 45,
 430100: 46,
 430200: 47,
 430300: 48,
 430400: 49,
 430500: 50,
 430999: 51,
 310120020: 52,
 310120030: 53,
 310120110: 54,
 310130030: 55,
 310130040: 56,
 310130080: 57,
 310150010: 58,
 310150030: 59,
 310150040: 60,
 310150070: 61,
 310150080: 62,
 310150090: 63,
 310150999: 64,
 310200100: 65,
 310200200: 66,
 310220100: 67,
 310220200: 68,
 310220300: 69,
 310220998: 70,
 310260100: 71,
 310260200: 72,
 310260300: 73,
 31

In [7]:
# 카테고리 df 빈 값 채워주기

cate = cate.fillna('None')
cate

,cate_enco,id,cat1,cat2,cat3
0,0,310,여성의류,None,None
1,1,320,남성의류,None,None
2,2,400,패션 액세서리,None,None
3,3,405,신발,None,None
4,4,421,시계,None,None
...,...,...,...,...,...
194,194,430200400,가방/지갑,남성가방,백팩
195,195,430200500,가방/지갑,남성가방,브리프케이스
196,196,430200999,가방/지갑,남성가방,기타 남성가방
197,197,430300100,가방/지갑,여행용 가방,캐리어


In [8]:
# 라벨인코딩한 카테고리id와 카테고리명 저장해놓은 딕셔너리

cate_dict = {}

for idx,row in cate.iterrows():
    if row['cat2'] == 'None':
        cate_dict[row.cate_enco] = row.cat1
    elif row['cat3'] == 'None':
        cate_dict[row.cate_enco] = row.cat1+' - '+row.cat2
    else:
        cate_dict[row.cate_enco] = row.cat1+' - '+row.cat2+' - '+row.cat3

cate_dict

{0: '여성의류',
 1: '남성의류',
 2: '패션 액세서리',
 3: '신발',
 4: '시계',
 5: '쥬얼리',
 6: '가방/지갑',
 7: '여성의류 - 원피스',
 8: '여성의류 - 치마',
 9: '여성의류 - 바지',
 10: '여성의류 - 언더웨어/홈웨어',
 11: '여성의류 - 테마/이벤트',
 12: '여성의류 - 점프수트',
 13: '여성의류 - 상의',
 14: '여성의류 - 아우터',
 15: '여성의류 - 셋업/세트',
 16: '남성의류 - 바지',
 17: '남성의류 - 언더웨어/홈웨어',
 18: '남성의류 - 테마/이벤트',
 19: '남성의류 - 상의',
 20: '남성의류 - 아우터',
 21: '남성의류 - 점프수트',
 22: '남성의류 - 셋업/세트',
 23: '패션 액세서리 - 모자',
 24: '패션 액세서리 - 안경/선글라스',
 25: '패션 액세서리 - 우산/양산',
 26: '패션 액세서리 - 벨트',
 27: '패션 액세서리 - 목도리/장갑',
 28: '패션 액세서리 - 스카프/넥타이',
 29: '패션 액세서리 - 양말/스타킹',
 30: '패션 액세서리 - 키링/키케이스',
 31: '패션 액세서리 - 기타 액세서리',
 32: '신발 - 스니커즈',
 33: '신발 - 여성화',
 34: '신발 - 남성화',
 35: '신발 - 스포츠화',
 36: '시계 - 여성시계',
 37: '시계 - 남성시계',
 38: '시계 - 시계용품',
 39: '쥬얼리 - 목걸이/펜던트',
 40: '쥬얼리 - 귀걸이/피어싱',
 41: '쥬얼리 - 팔찌',
 42: '쥬얼리 - 발찌',
 43: '쥬얼리 - 반지',
 44: '쥬얼리 - 쥬얼리 세트',
 45: '쥬얼리 - 기타 쥬얼리',
 46: '가방/지갑 - 여성가방',
 47: '가방/지갑 - 남성가방',
 48: '가방/지갑 - 여행용 가방',
 49: '가방/지갑 - 여성지갑',
 50: '가방/지갑 - 남성지갑',
 51: '가방/지갑

In [ ]:
# 현재 데이터
data

,id,name,cat_id
0,207604546,아쿠비클럽 숏패딩,310300200
1,172324242,듀베티카 이탈리아 브랜드 경량 패딩,310300200
2,216829622,"자라 러버라이즈드 패딩점퍼 xs 샌드(세탁1회가 전부,착용안함,새상품)",310300200
3,208445966,아이보리 퀼팅 패딩,310300200
4,214399932,Kenneth Cole 케네스콜 미들 다운패딩 S,310300200
...,...,...,...
156995,217819703,"플라워, 곰 키링, 키홀더, 열쇠고리",400999
156996,174817306,쿠로미핀,400999
156997,225929172,지방시 U 패드락,400999
156998,204684604,고양이 쿠키 브로치/페이크스위츠/로리타/카와이스타일/미니어쳐/스위티시모,400999


In [9]:
data['cate_enco'] = 0
for idx, row in data.iterrows():
    label_name = cate_num[row['cat_id']]
    data.iloc[idx, 3] = label_name
data = data.drop(['cat_id'], axis=1)
data

,id,name,cate_enco
0,207604546,아쿠비클럽 숏패딩,80
1,172324242,듀베티카 이탈리아 브랜드 경량 패딩,80
2,216829622,"자라 러버라이즈드 패딩점퍼 xs 샌드(세탁1회가 전부,착용안함,새상품)",80
3,208445966,아이보리 퀼팅 패딩,80
4,214399932,Kenneth Cole 케네스콜 미들 다운패딩 S,80
...,...,...,...
156995,217819703,"플라워, 곰 키링, 키홀더, 열쇠고리",31
156996,174817306,쿠로미핀,31
156997,225929172,지방시 U 패드락,31
156998,204684604,고양이 쿠키 브로치/페이크스위츠/로리타/카와이스타일/미니어쳐/스위티시모,31


In [ ]:
len(data['cate_enco'].unique())

157

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157000 entries, 0 to 156999
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   id         157000 non-null  int64 
 1   name       157000 non-null  object
 2   cate_enco  157000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [10]:
data_list = []
for q, label in zip(data['name'], data['cate_enco']):
    data1 = []
    data1.append(q)
    data1.append(label)

    data_list.append(data1)

In [ ]:
data_list[0] # 이전 코드로 확인

['노스페이스 롱패딩', 310090050]

In [ ]:
data_list[0] # 현재 코드

['아쿠비클럽 숏패딩', 0]

# Train data & Test data

In [11]:
# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=25, stratify=[data[1] for data in data_list])

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

125600
31400


# KoBERT 입력 데이터로 만들기

In [12]:
from transformers import BertTokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [torch.tensor(i[label_idx], dtype=torch.long) for i in dataset]
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair

    def __getitem__(self, i):
        sentence = self.sentences[i]
        label = self.labels[i]

        inputs = self.tokenizer.encode_plus(
            sentence,
            None if not self.pair else sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length' if self.pad else None,
            truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        valid_length = torch.tensor(len(inputs['input_ids']), dtype=torch.long)

        return (input_ids, attention_mask, valid_length, label)

    def __len__(self):
        return len(self.labels)


In [13]:
# Setting parameters
max_len = 64
batch_size = 48
warmup_ratio = 0.1
num_epochs = 10      # 5일 때 정확도 train: 0.79, test: 0.74 - 1시간 52분?
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
num_classes = len(cate)

In [14]:
# 토큰화
tokenizer = get_tokenizer()
tok = tokenizer

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [ ]:
data_train[0]

(tensor([   2,  517, 7776, 5103, 6079, 5538, 6028, 3417, 6079, 6003, 2929, 7559,
         6855, 4384, 5400, 7773,  365,  119,  385,  420,  430,    3,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(64),
 tensor(159))

In [ ]:
data_test[0]

(tensor([   2, 1423, 2872, 7234, 7941, 4829, 5770, 6079, 7706,  553,  176,  517,
         7692, 5772,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(64),
 tensor(135))

In [ ]:
# 이전 코드로 확인
dataset_train[0] # [CLS], "화이트", "##무드", "중청", "##바", "##지", [SEP]

['화이트무드 중청바지', 310140030]

In [ ]:
dataset_train[0]

['큐빅셋팅 핸드메이드 귀걸이 악세사리', 117]

In [15]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

# KoBERT 학습모델 만들기

In [ ]:
len(data['cate_enco'].unique()) # 카테고리 수 157개

157

In [16]:
class BERTClassifier(nn.Module): # for 번개장터 (메인)
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 199,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert                # pretrained_bert 모델
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)

        return output


In [17]:
# GPU

from torch.optim import AdamW

# BERT 모델 불러오기
kobert_model = get_kobert_model()
model = BERTClassifier(kobert_model, num_classes=num_classes, dr_rate=0.5).to(device)

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
optimizer = AdamW(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss().to(device)

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc


In [ ]:
## CPU

# BERT 모델 불러오기
# kobert_model = get_kobert_model()
# model = BERTClassifier(kobert_model,  dr_rate=0.5)
# # optimizer와 schedule 설정
# no_decay = ['bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
# ]

# optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss()

# t_total = len(train_dataloader) * num_epochs
# warmup_step = int(t_total * warmup_ratio)

# scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# # 정확도 측정을 위한 함수 정의
# def calc_accuracy(X, Y):
#     max_vals, max_indices = torch.max(X, 1)
#     train_acc = (max_indices.cpu() == Y.cpu()).sum().data.numpy() / max_indices.size()[0]
#     return train_acc


# KoBERT 모델 학습시키기

In [ ]:
for i, (data1, data2, data3, data4) in enumerate(train_dataloader):
  print(data1)
  print(data2)
  print(data3)
  print(data4)
  break

tensor([[   2,  517, 7776,  ...,    1,    1,    1],
        [   2,  542,  415,  ...,    1,    1,    1],
        [   2, 3289, 6438,  ...,    1,    1,    1],
        ...,
        [   2, 2333, 7096,  ...,    1,    1,    1],
        [   2, 1397, 7573,  ...,    1,    1,    1],
        [   2, 2355, 6579,  ...,    1,    1,    1]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64,
        64, 64, 64, 64, 64, 64, 64, 64, 64, 64])
tensor([159, 161,  58, 122,  32,  50, 137, 195, 107,  69, 177,  88,  80, 122,
         25, 181,  32, 129, 131, 197,  88,  12, 138, 136, 192, 131, 105, 185,
         49

In [ ]:
# 테스트 데이터로 에폭5 해서 구글드라이브에 저장 완료
# 에폭10으로 해보는 중 ~~~~ 4시간

from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)
        out = model(token_ids, attention_mask)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.long().to(device)
        out = model(token_ids, attention_mask)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


  0%|          | 0/2617 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 5.257046222686768 train acc 0.020833333333333332
epoch 1 batch id 201 loss 5.216948509216309 train acc 0.004664179104477613
epoch 1 batch id 401 loss 5.257583141326904 train acc 0.00618246051537822
epoch 1 batch id 601 loss 5.1299920082092285 train acc 0.006932889628397127
epoch 1 batch id 801 loss 4.7815704345703125 train acc 0.01149604660840616
epoch 1 batch id 1001 loss 4.452646732330322 train acc 0.030927405927405904
epoch 1 batch id 1201 loss 3.892627716064453 train acc 0.06265611990008327
epoch 1 batch id 1401 loss 3.441695213317871 train acc 0.09713300023792532
epoch 1 batch id 1601 loss 2.6915862560272217 train acc 0.12947636893608155
epoch 1 batch id 1801 loss 2.39583420753479 train acc 0.15928650749583553
epoch 1 batch id 2001 loss 2.6860907077789307 train acc 0.18601116108612345
epoch 1 batch id 2201 loss 1.7232613563537598 train acc 0.210519839466909
epoch 1 batch id 2401 loss 1.6142578125 train acc 0.23322747466333477
epoch 1 batch id 2601 loss 1.68

  0%|          | 0/655 [00:00<?, ?it/s]

epoch 1 test acc 0.5484096692111958


  0%|          | 0/2617 [00:00<?, ?it/s]

...

epoch 9 test acc 0.7486323155216273


  0%|          | 0/2617 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.3735952079296112 train acc 0.8958333333333334
epoch 10 batch id 201 loss 0.5392159223556519 train acc 0.8995646766169159
epoch 10 batch id 401 loss 0.2588355243206024 train acc 0.9018599334995855
epoch 10 batch id 601 loss 0.1951615810394287 train acc 0.904360787576262
epoch 10 batch id 801 loss 0.4021952152252197 train acc 0.9043643362463589
epoch 10 batch id 1001 loss 0.29183724522590637 train acc 0.9060730935730938
epoch 10 batch id 1201 loss 0.2885722517967224 train acc 0.9075423258395784
epoch 10 batch id 1401 loss 0.4017922878265381 train acc 0.9073132286462056
epoch 10 batch id 1601 loss 0.2629799246788025 train acc 0.907648865292527
epoch 10 batch id 1801 loss 0.2822180688381195 train acc 0.9075976309457727
epoch 10 batch id 2001 loss 0.2794937789440155 train acc 0.9082542062302201
epoch 10 batch id 2201 loss 0.18266178667545319 train acc 0.9083749810692126
epoch 10 batch id 2401 loss 0.31995582580566406 train acc 0.9083454810495644
epoch 10 batch id 

  0%|          | 0/655 [00:00<?, ?it/s]

epoch 10 test acc 0.7494274809160297


In [ ]:
# 모델 저장 경로 지정
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model.pth"

# 모델 저장
torch.save(model.state_dict(), save_path)

In [18]:
# 저장된 모델 불러오기

## GPU
model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model_kobert10.pth"
model.load_state_dict(torch.load(save_path))


## CPU
# device = torch.device("cpu")
# model = BERTClassifier(kobert_model, dr_rate=0.5)
# save_path = "/content/drive/MyDrive/KDT/Final Project/modeling/model_kobert_epoch5.pth"
# model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))


FileNotFoundError: ignored

# 새로운 문장 테스트

In [ ]:
# 새 코드

import numpy as np
from kobert_transformers import get_tokenizer

tokenizer = get_tokenizer()
tok = get_tokenizer()

# 카테고리와 인덱스 매핑
# category_map = {}
# for i in range(174):
#     category_map[i] = f"카테고리{i+1}"

def predict(predict_sentence):
    category_id = 0  # 예측할 카테고리의 인덱스 값
    data = [(predict_sentence, category_id)]  # 상품명, 카테고리id
    dataset_another = BERTDataset(data, sent_idx=0, label_idx=1, bert_tokenizer=tok, max_len=max_len, pad=True, pair=False)
    test_dataloader = torch.utils.data.DataLoader(dataset_another, batch_size=1, num_workers=0)  # 배치 크기를 1로 설정

    model.eval()

    for batch_id, (input_ids, attention_mask, valid_length, label) in enumerate(test_dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        valid_length = valid_length.to(device)
        label = label.to(device)

        out = model(input_ids, attention_mask)

        test_eval = []
        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) in cate_dict:
                category = cate_dict[np.argmax(logits)]
                test_eval.append(category)

        if test_eval:
            print(">> 입력하신 상품명으로부터 예측된 카테고리:", test_eval[0])
        else:
            print(">> 카테고리를 예측할 수 없습니다.")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


# 학습1에서 입력해보기!!

In [ ]:
# 질문 무한 반복! 0 입력시 종료
while True:
    sentence = input("상품명을 입력해주세요 : ")
    if sentence == '0':
        break
    predict(sentence)
    print("\n")

상품명을 입력해주세요 : 롱패딩
>> 입력하신 상품명으로부터 예측된 카테고리: 여성의류 - 아우터 - 패딩


상품명을 입력해주세요 : 새 롱패딩 팔아요
>> 입력하신 상품명으로부터 예측된 카테고리: 여성의류 - 아우터 - 패딩


상품명을 입력해주세요 : 1회 착용 귀여운 귀걸이
>> 입력하신 상품명으로부터 예측된 카테고리: 쥬얼리 - 귀걸이/피어싱 - 패션 귀걸이


상품명을 입력해주세요 : 새 야구화
>> 입력하신 상품명으로부터 예측된 카테고리: 신발 - 스포츠화 - 야구화


상품명을 입력해주세요 : 기스 조금 있는 시계
>> 입력하신 상품명으로부터 예측된 카테고리: 시계 - 시계용품 - 기타 시계용품


상품명을 입력해주세요 : 0


# 학습2 - Model 1 + Dense Layer (네이버 쇼핑 데이터로 학습) 가중치 튜닝

In [19]:
# 네이버 데이터

# 구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
nvdata = pd.read_csv('/content/drive/MyDrive/KDT/Final Project/data/sample_naver.csv')
nvdata

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,name,cat_id
0,85966336409,캠핑룩 청 반팔 멜빵바지 점프 슈트 작업복 수트 Amekaji 멀티 포켓 야외 레트로 캐,320400
1,83441837670,남자 데님 점프수트 빅사이즈 가을 바디프로필 의상 컨셉 바프 츠나기 점프수트,320400
2,40182258929,저스트앳 남성용 라텍스 점프수트 PVC 지퍼 바디수트 섹시한 오픈 가랑이 클럽웨어 ...,320400
3,85966336410,캠핑룩 청 반팔 멜빵바지 점프 슈트 작업복 수트 BlingBlingee 여름 하이 웨이스,320400
4,40565489373,남성 점프수트 아우터 Maden-빈티지 오버올 남성용 카고 작업 바지 배기 번호판 ...,320400
...,...,...,...
132995,40227277136,알루미늄 프레임 캐리어 18인치 20인치 단거리 출장 소형 기내용 다용도 여행가방,430300100
132996,85868871319,가성비 새로운 여행 롤링 가방 PU 수하물 여성용 바퀴 달린 캐빈 트롤리,430300100
132997,86144541686,쌤소나이트 아셀라 3.0 미디움 익스펜더블 스피너,430300100
132998,40140685022,여성용 가벼운 캐리어 가방 경량 여행 저소음 여성,430300100


In [ ]:
len(nvdata['cat_id'].unique())

133

## 입력데이터로 만들기

In [20]:
# 배치 단위로 나누기

# 입력 데이터 형태로 고치기

# nvdata = nvdata.drop(['image_url','low_price'], axis=1)

nvdata['cate_enco'] = 0
for idx, row in nvdata.iterrows():
    label_name = cate_num[row['cat_id']]
    nvdata.iloc[idx, 3] = label_name
nvdata = nvdata.drop(['cat_id'], axis=1)

nvdata_list = []
for q, label in zip(nvdata['name'], nvdata['cate_enco']):
    data1 = []
    data1.append(q)
    data1.append(label)

    nvdata_list.append(data1)

# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(nvdata_list, test_size=0.2, random_state=25, stratify=[data[1] for data in nvdata_list])

from transformers import BertTokenizer

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        self.sentences = [i[sent_idx] for i in dataset]
        self.labels = [torch.tensor(i[label_idx], dtype=torch.long) for i in dataset]
        self.tokenizer = bert_tokenizer
        self.max_len = max_len
        self.pad = pad
        self.pair = pair

    def __getitem__(self, i):
        sentence = self.sentences[i]
        label = self.labels[i]

        inputs = self.tokenizer.encode_plus(
            sentence,
            None if not self.pair else sentence,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length' if self.pad else None,
            truncation=True
        )

        input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
        attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
        valid_length = torch.tensor(len(inputs['input_ids']), dtype=torch.long)

        return (input_ids, attention_mask, valid_length, label)

    def __len__(self):
        return len(self.labels)

# Setting parameters
max_len = 64
batch_size = 48
warmup_ratio = 0.1
num_epochs = 10      # 5일 때 정확도 train: 0.61, test: 0.60 - 42분  ## 3일 때 정확도 train: 0.47, test: 0.51 - 17분
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
num_classes = len(nvdata)

# 토큰화
tokenizer = get_tokenizer()
tok = tokenizer

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


## 학습1 BERT 모델 불러오기

In [ ]:
# 학습1 모델 로드

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 146,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert # kobert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate is not None:
            self.dropout = nn.Dropout(p=dr_rate)
        else:
            self.dropout = None

    def forward(self, input_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask,
                                    return_dict=False)
        output = self.dropout(pooled_output)
        output = self.classifier(output)

        return output

# BERT 모델 불러오기
kobert_model = get_kobert_model()

# 저장된 모델 불러오기

## GPU
model = BERTClassifier(kobert_model, dr_rate=0.5).to(device)
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model_kobert_epoch5.pth"

model.load_state_dict(torch.load(save_path, map_location=device))


<All keys matched successfully>

## 학습2 모델 생성

In [21]:
# BERT 모델의 아키텍처 뒤에 Dense 레이어 추가 (학습2 모델 생성)

class BERTClassifierforNaver(nn.Module):  # 네이버 미세조정을 위한 모델
    def __init__(self,
                  bert,
                  hidden_size = 199, # 번개장터 클래스 개수 (output이 199으로 나오니까)
                  num_classes = 133,   # 네이버 카테고리 개수
                  dr_rate=None,
                  params=None):
            super(BERTClassifierforNaver, self).__init__()
            self.bert = bert  # 위에서 학습시킨 번개장터 모델 : 아웃풋은 하나(벡터)
            self.dr_rate = dr_rate

            self.classifier = nn.Linear(hidden_size, num_classes)
            if dr_rate is not None:
                self.dropout = nn.Dropout(p=dr_rate)
            else:
                self.dropout = None

    def forward(self, input_ids, attention_mask):

        output = self.bert(input_ids=input_ids,
                                    attention_mask=attention_mask)
        output = self.dropout(output)
        output = self.classifier(output)

        return output

# GPU

from torch.optim import AdamW

# 학습2 모델 불러오기
model_nv = BERTClassifierforNaver(model, num_classes=num_classes, dr_rate=0.5).to(device)


# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(model.parameters(), lr=learning_rate)

loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc



## 가중치 튜닝 모델 학습

In [22]:
# 가중치 튜닝 학습

from tqdm.notebook import tqdm

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model_nv.train()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.to(device)
        attention_mask = attention_mask.to(device)
        label = label.to(device)
        out = model_nv(token_ids, attention_mask)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_nv.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model_nv.eval()
    for batch_id, (token_ids, attention_mask, valid_length, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        label = label.long().to(device)
        out = model_nv(token_ids, attention_mask)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))


  0%|          | 0/2217 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 11.915648460388184 train acc 0.0
epoch 1 batch id 201 loss 11.784561157226562 train acc 0.0
epoch 1 batch id 401 loss 11.604182243347168 train acc 0.0
epoch 1 batch id 601 loss 11.554211616516113 train acc 0.0
epoch 1 batch id 801 loss 11.47604751586914 train acc 5.2018310445276733e-05
epoch 1 batch id 1001 loss 11.072274208068848 train acc 0.00020812520812520813
epoch 1 batch id 1201 loss 10.563962936401367 train acc 0.0007979461559811271
epoch 1 batch id 1401 loss 9.908147811889648 train acc 0.0027510111824886993
epoch 1 batch id 1601 loss 9.740696907043457 train acc 0.005465334166146157
epoch 1 batch id 1801 loss 9.156659126281738 train acc 0.00961271515824542
epoch 1 batch id 2001 loss 9.040209770202637 train acc 0.015554722638680645
epoch 1 batch id 2201 loss 7.600185394287109 train acc 0.024316598515826147
epoch 1 train acc 0.025094910539768468


  0%|          | 0/555 [00:00<?, ?it/s]

epoch 1 test acc 0.4317192192192191


  0%|          | 0/2217 [00:00<?, ?it/s]

...

epoch 9 test acc 0.8811561561561562


  0%|          | 0/2217 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.3630015552043915 train acc 0.9166666666666666
epoch 10 batch id 201 loss 0.3528759181499481 train acc 0.9451699834162524
epoch 10 batch id 401 loss 0.6407053470611572 train acc 0.9462281795511224
epoch 10 batch id 601 loss 0.42504584789276123 train acc 0.9452648363838047
epoch 10 batch id 801 loss 0.6393887996673584 train acc 0.9446265085310027
epoch 10 batch id 1001 loss 0.688107967376709 train acc 0.944555444555444
epoch 10 batch id 1201 loss 0.1443440467119217 train acc 0.9450284485151256
epoch 10 batch id 1401 loss 0.03825276345014572 train acc 0.9457084225553172
epoch 10 batch id 1601 loss 0.33175039291381836 train acc 0.945554861544867
epoch 10 batch id 1801 loss 0.27169567346572876 train acc 0.945770868036276
epoch 10 batch id 2001 loss 0.6746900081634521 train acc 0.945829168748958
epoch 10 batch id 2201 loss 0.19058392941951752 train acc 0.9455550507345133
epoch 10 train acc 0.9456613667117715


  0%|          | 0/555 [00:00<?, ?it/s]

epoch 10 test acc 0.8808558558558562


In [23]:
# 모델 저장 경로 지정
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model.pth"

# 모델 저장
torch.save(model_nv.state_dict(), save_path)

In [ ]:
# 학습2 모델 불러오기

model_nv = BERTClassifierforNaver(model, num_classes=num_classes, dr_rate=0.5).to(device) # 네이버쇼핑 학습2 모델
save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/model_kobert10.pth"
model.load_state_dict(torch.load(save_path))

In [ ]:
# 최종모델

model = BERTClassifier(kobert_model, dr_rate=0.5).to(device) # 번개장터 학습1 모델

# 학습2 모델 불러오기
model_nv = BERTClassifierforNaver(model, num_classes=num_classes, dr_rate=0.5).to(device) # 네이버쇼핑 학습2 모델

save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/kobert_nv_epoch5.pth" # 저장경로

model_nv.load_state_dict(torch.load(save_path, map_location=device)) # 모델 가중치 저장

model_final = model_nv.bert # 번개장터 모델만 빼오기(네이버쇼핑 분류모델 제거) # 최종모델

save_path = "/content/drive/MyDrive/KDT/Final Project/modeling_data/kobert_final_epoch5.pth" # 저장경로

model_final.load_state_dict(torch.load(save_path, map_location=device)) # 최종모델 저장